In [0]:
#Load Config and Setup Enviorment Variables
# state_under_test = "paymentPending"
state_under_test = "appealSubmitted"
# state_under_test = "awaitingRespondentEvidence(a)"
# state_under_test = "awaitingRespondentEvidence(b)"


from pyspark.sql import functions as F

config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env_name = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()
 
# print(f"env_code: {lz_key}")  # This won't be redacted
# print(f"env_name: {env_name}")  # This won't be redacted
 
KeyVault_name = f"ingest{lz_key}-meta002-{env_name}"
# print(f"KeyVault_name: {KeyVault_name}")
 
# Service principal credentials
client_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-ID")
client_secret = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-SECRET")
tenant_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-TENANT-ID")
 
# Storage account names
curated_storage = f"ingest{lz_key}curated{env_name}"
checkpoint_storage = f"ingest{lz_key}xcutting{env_name}"
raw_storage = f"ingest{lz_key}raw{env_name}"
landing_storage = f"ingest{lz_key}landing{env_name}"
external_storage = f"ingest{lz_key}external{env_name}"
  
# Spark config for curated storage (Delta table)
spark.conf.set(f"fs.azure.account.auth.type.{curated_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{curated_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{curated_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{curated_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{curated_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{checkpoint_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{checkpoint_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{checkpoint_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{checkpoint_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{checkpoint_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{raw_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{raw_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{raw_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{raw_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{raw_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{landing_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{landing_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{landing_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{landing_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{landing_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
 
 
# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{external_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{external_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{external_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{external_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{external_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
  
# Setting variables for use in subsequent cells
bronze_path = f"abfss://bronze@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/"
silver_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/"
audit_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/AUDIT/{state_under_test}"
gold_path = f"abfss://gold@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS/{state_under_test}"
 
 
# Print all variables
# variables = {
#     # "read_hive": read_hive,
    
#     "bronze_path": bronze_path,
#     "silver_path": silver_path,
#     "audit_path": audit_path,
#     "gold_path": gold_path,
#     "key_vault": KeyVault_name,
#     "AppealState": state_under_test
 
# }
 
# display(variables)

import json

#Get Latest Json Folder
json_location = dbutils.fs.ls(f"{gold_path}/")[-1]
latest_json_location = json_location.name
dbutils.fs.ls(f"{gold_path}/{latest_json_location}")

#Set Paths
try: 
    json_path = f"{gold_path}/{latest_json_location}/JSON/"
    json_failed_path = f"{gold_path}/{latest_json_location}/INVALID_JSON/"
    M1_silver = f"{silver_path}/silver_appealcase_detail"
    M1_bronze = f"{bronze_path}/bronze_appealcase_crep_rep_floc_cspon_cfs"
    M2_bronze = f"{bronze_path}/bronze_appealcase_caseappellant_appellant"
    M2_silver = f"{silver_path}/silver_caseapplicant_detail"
    M3_silver = f"{silver_path}/silver_status_detail"
    C = f"{silver_path}/silver_appealcategory_detail"
    bhc = f"{bronze_path}/bronze_hearing_centres"
    bat = f"{bronze_path}/bronze_appealtype" 
    docsr = f"{bronze_path}/bronze_documentsreceived"   
    apl_audit = f"{audit_path}/apl_active_payment_pending_cr_audit_table/"
    sh =  f"{silver_path}/silver_history_detail"
except:
    print(f"Error during fetch: {str(e)}")

#Create and Load Dataframes
json_data = spark.read.format("json").load(json_path)
json_failed_data = spark.read.format("json").load(json_failed_path)
M1_silver = spark.read.format("delta").load(M1_silver)
M1_bronze = spark.read.format("delta").load(M1_bronze)
M2_bronze = spark.read.format("delta").load(M2_bronze)
M2_silver = spark.read.format("delta").load(M2_silver)
M3_silver = spark.read.format("delta").load(M3_silver)
C = spark.read.format("delta").load(C)
bhc = spark.read.format("delta").load(bhc)
bat = spark.read.format("delta").load(bat)
docsr = spark.read.format("delta").load(docsr)
apl_audit = spark.read.format("delta").load(apl_audit)
sh_audit = spark.read.format("delta").load(sh)

#Can be removed later, added to allow developing of code in this notebook to begin with before moving to func files
from pyspark.sql.functions import (
    col, when, lit, array, struct, collect_list, 
    max as spark_max, date_format, row_number, expr, 
    size, udf, coalesce, concat_ws, concat, trim, year, split, datediff,
    collect_set, current_timestamp,transform, first, array_contains
)

# Data Patching - For Data Cut : 07-04-2025

In [0]:
###############################
#UPDATE BRONZE DATA SCRIPT FOR PAYMENT PENDING.
#
#NOTE: The below code will update bronze data that will not pass the DQ expecation checks due to
#issues in the data that will be resolved before live but are needeed to get all the data through the checks
#and sent to CCD in the mean time
###############################
from pyspark.sql.functions import *
from delta.tables import DeltaTable

######################
#TO FIX PP DATA FROM FIRST STAGING DATA CUT (now superseeded due to new data cut)
######################
# BirthDate / appellantDateOfBirth

# bronze_table = DeltaTable.forName(spark,"ariadm_active_appeals.bronze_appealcase_caseappellant_appellant")

# display(bronze_table.toDF().filter(col("CaseNo").isin("HU/00278/2025", "HU/00455/2025", "HU/00472/2025" )).select("CaseNo", "BirthDate"))

# bronze_table.update(
#     condition=col("CaseNo").isin("HU/00278/2025", "HU/00455/2025", "HU/00472/2025"),
#     set={"BirthDate": lit("2000-02-01T00:00:00Z")}
# )

# display(bronze_table.toDF().filter(col("CaseNo").isin("HU/00278/2025", "HU/00455/2025", "HU/00472/2025" )).select("CaseNo", "BirthDate"))


# #################
# #valid_appellantNationalitiesDescription_not_null
# #and
# #valid_appellantNationalities_not_null
# #NationalityId
# #Where No mapping required for 201/203

# bronze_table = DeltaTable.forName(spark, "ariadm_active_appeals.bronze_appealcase_crep_rep_floc_cspon_cfs")

# display(bronze_table.toDF().filter(col("CaseNo").isin("HU/00302/2025", "HU/00569/2025", "HU/00586/2025","HU/00560/2025" )).select("CaseNo", "NationalityId"))

# bronze_table.update(
#     condition=col("CaseNo").isin("HU/00302/2025", "HU/00569/2025", "HU/00586/2025","HU/00560/2025"),
#     set={"NationalityId": lit("41")}
# )

# display(bronze_table.toDF().filter(col("CaseNo").isin("HU/00302/2025", "HU/00569/2025", "HU/00586/2025","HU/00560/2025" )).select("CaseNo", "NationalityId"))

# #################
# #valid_oocAddressLine1 valid_oocAddressLine2
# #changing null values to actual values

# bronze_table = DeltaTable.forName(spark, "ariadm_active_appeals.bronze_appealcase_crep_rep_floc_cspon_cfs")
# display(bronze_table.toDF().filter(col("CaseNo").isin("HU/00185/2025", "HU/02151/2024")).select("CaseNo", "CaseRep_Address1", "CaseRep_Address2", "CaseRep_Address3", "CaseRep_Address4" , "CaseRep_Address5", "CaseRep_Postcode"))

# bronze_table.update(
#     condition=col("CaseNo").isin("HU/00185/2025", "HU/02151/2024"),
#     set={"CaseRep_Address1": lit("925 Lisa Plains Apt. 642X"),
#          "CaseRep_Address2" : lit("Hill SquareX"),
#         "CaseRep_Address3" : lit("LynchhavenX"),
#         "CaseRep_Address4" : lit("AustraliaX"),
#         "CaseRep_Address5" : lit("NLX"),
#         # "CaseRep_Postcode" : lit("Hill SquareX"),
         
#          }
# )


# display(bronze_table.toDF().filter(col("CaseNo").isin("HU/00185/2025", "HU/02151/2024")).select("CaseNo", "CaseRep_Address1", "CaseRep_Address2", "CaseRep_Address3", "CaseRep_Address4" , "CaseRep_Address5", "CaseRep_Postcode"))


# #################
# #valid_oocAddressLine1 valid_oocAddressLine2
# #change in ooc4 ReunionX to GuamX

# bronze_table = DeltaTable.forName(spark, "ariadm_active_appeals.bronze_appealcase_crep_rep_floc_cspon_cfs")
# display(bronze_table.toDF().filter(col("CaseNo").isin("HU/02191/2024", "HU/01475/2024")).select("CaseNo", "CaseRep_Address1", "CaseRep_Address2", "CaseRep_Address3", "CaseRep_Address4" , "CaseRep_Address5", "CaseRep_Postcode"))

# bronze_table.update(
#     condition=col("CaseNo").isin("HU/01475/2024", "HU/02191/2024"),
#     set={
#         "CaseRep_Address4" : lit("AustraliaX")                 
#          }
# )


# display(bronze_table.toDF().filter(col("CaseNo").isin("HU/02191/2024", "HU/01475/2024")).select("CaseNo", "CaseRep_Address1", "CaseRep_Address2", "CaseRep_Address3", "CaseRep_Address4" , "CaseRep_Address5", "CaseRep_Postcode"))



# ################################

# cases_to_update = ['EA/02806/2023',
# 'HU/00575/2025',
# 'HU/00581/2025',
# 'HU/00447/2025',
# 'HU/00574/2023',
# 'HU/00591/2025',
# 'EA/00588/2025',
# 'EA/00551/2025',
# 'HU/00304/2025',
# 'EA/00495/2025',
# 'EA/00560/2025',
# 'EA/06826/2022',
# 'EA/00490/2025',
# 'EA/00554/2025',
# 'EA/01778/2024',
# 'HU/00562/2025',
# 'EA/00552/2025',
# 'HU/00511/2025',
# 'EA/00483/2025',
# 'EA/09676/2022',
# 'EA/00493/2025',
# 'HU/00224/2025',
# 'EA/00496/2025',
# 'EA/00538/2025',
# 'EA/00558/2025',
# 'EA/08372/2022',
# 'HU/00822/2024',
# 'HU/00574/2025',
# 'EA/02065/2024',
# 'HU/00442/2025',
# 'EA/00586/2025',
# 'HU/00590/2025',
# 'HU/00569/2025',
# 'EA/00557/2025',
# 'HU/02346/2024',
# 'EA/00562/2025',
# 'HU/00573/2025',
# 'HU/00571/2025',
# 'EA/01893/2023',
# 'EA/00584/2025',
# 'HU/00579/2025',
# 'HU/00555/2025',
# 'HU/00583/2025',
# 'EA/00591/2025',
# 'EA/00556/2025',
# 'EA/00497/2025',
# 'HU/01972/2023',
# 'EA/00437/2025',
# 'HU/00577/2025',
# 'EA/00585/2025',
# 'HU/00252/2025',
# 'HU/00557/2025',
# 'EA/00485/2025',
# 'HU/00563/2025',
# 'HU/00278/2025',
# 'EA/00559/2025',
# 'EA/00553/2025',
# 'HU/00578/2025',
# 'HU/00445/2025',
# 'HU/00325/2025',
# 'EA/00555/2025',
# 'HU/00572/2025',
# 'HU/00582/2025',
# 'EA/00587/2025',
# 'HU/00638/2024',
# 'HU/00453/2025',
# "EA/00289/2025"
# ]


# bronze_table = DeltaTable.forName(spark, "ariadm_active_appeals.bronze_appealcase_caseappellant_appellant")

# display(bronze_table.toDF().filter(col("CaseNo").isin(cases_to_update)).select("CaseNo", "Appellant_Address4"))

# bronze_table.update(
#     condition=col("CaseNo").isin(cases_to_update),
#     set={
#         "Appellant_Address4" : lit("AustraliaX")                 
#          }
# )


# display(bronze_table.toDF().filter(col("CaseNo").isin(cases_to_update)).select("CaseNo", "Appellant_Address4"))




# Data Patching - For Data cut from : 02-02-2026

In [0]:
#TABLES
# bronze_table_to_update = "ariadm_active_appeals.bronze_appealcase_crep_rep_floc_cspon_cfs"
# bronze_table_to_update = "ariadm_active_appeals.bronze_appealcase_caseappellant_appellant"

# FROM paymentpending_gold.stg_main_paymentPending_validation
# FROM paymentpending_gold.stg_main_appeal_submitted_validation

In [0]:
##############################
#valid_countryGovUkOocAdminJ - patch a postcode in "Appellant_Postcode"
##############################


#Setup
checks = {}
bronze_table_to_update = "ariadm_active_appeals.bronze_appealcase_caseappellant_appellant"

#Add the Gold Tables to the list below for any States that need updating
state_to_update_1 =  "paymentpending_gold.stg_main_paymentPending_validation"
state_to_update_2 = "paymentpending_gold.stg_main_appeal_submitted_validation"
# states_to_update = [state_to_update_1, state_to_update_2]
states_to_update = [state_to_update_2]




checks["valid_countryGovUkOocAdminJ"] = ("(((array_contains(valid_categoryIdList, 38)) AND (countryGovUkOocAdminJ IS NOT NULL) AND (countryGovUkOocAdminJ IN ('AF', 'AX', 'AL', 'DZ', 'AD', 'AO', 'AI', 'AG', 'AR', 'AM', 'AW', 'AC', 'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BM', 'BT', 'BO', 'BQ', 'BA', 'BW', 'BR', 'IO', 'VG', 'BN', 'BG', 'BF', 'BI', 'KH', 'CM', 'CA', 'IC', 'CV', 'KY', 'CF', 'EA', 'TD', 'CL', 'CN', 'CX', 'CO', 'KM', 'CD', 'CG', 'CK', 'CR', 'HR', 'CU', 'CW', 'CY', 'CZ', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE', 'ET', 'FK', 'FO', 'FJ', 'FI', 'FR', 'GF', 'PF', 'TF', 'GA', 'GM', 'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GP', 'GT', 'GN', 'GW', 'GY', 'HT', 'HN', 'HK', 'HU', 'IS', 'IN', 'ID', 'IR', 'IQ', 'IE', 'IL', 'IT', 'CI', 'JM', 'JP', 'JO', 'KZ', 'KE', 'KI', 'KO', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI', 'LT', 'LU', 'MO', 'MK', 'MG', 'YT', 'MW', 'MY', 'MV', 'ML', 'MT', 'MQ', 'MR', 'MU', 'MX', 'MD', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM', 'NA', 'NR', 'NF', 'NP', 'NL', 'NC', 'NZ', 'NI', 'NE', 'NG', 'NU', 'KP', 'NO', 'OM', 'PK', 'PW', 'PA', 'PG', 'PY', 'PE', 'PH', 'PN', 'PL', 'PT', 'PR', 'QA', 'RE', 'RO', 'RU', 'RW', 'SM', 'ST', 'SA', 'SN', 'RS', 'SC', 'SL', 'SG', 'SK', 'SI', 'SB', 'ZA', 'KR', 'SS', 'ES', 'LK', 'BQ', 'SH', 'KN', 'LC', 'MF', 'VC', 'SD', 'SR', 'SZ', 'SE', 'CH', 'SY', 'TW', 'TJ', 'TZ', 'TH', 'TL', 'TG', 'TK', 'TO', 'TT', 'TN', 'TR', 'TM', 'TC', 'TV', 'UG', 'UA', 'AE', 'GB', 'UY', 'US', 'UZ', 'VU', 'VA', 'VE', 'VN', 'WF', 'EH', 'WS', 'YE', 'ZM', 'ZW'))) OR (countryGovUkOocAdminJ IS NULL))")

#####################
#Get Cases Failing Validation
#####################
dq_rules = "({0})".format(" AND ".join(checks.values()))


for state_table in states_to_update:

    df_validation_filtered = spark.sql(f"""
    SELECT *
    FROM {state_table}
    WHERE CaseNo NOT IN (
    SELECT
        CaseNo
    FROM {state_table}
    WHERE {dq_rules})
    """)
    df_validation_filtered = df_validation_filtered.select("CaseNo", "countryGovUkOocAdminJ")


    #####################
    #join with bronze data
    #####################
    #Join Failed Record in Gold With Bronze
    df_validation_filtered = df_validation_filtered.join(M2_bronze, on="CaseNo", how="inner")
    display(df_validation_filtered.select("CaseNo",  "countryGovUkOocAdminJ", "Appellant_Postcode"))

    #####################
    #Update Data
    #####################
    #Pull out case Numbers to patch
    case_nos_to_update = [row["CaseNo"] for row in df_validation_filtered.select("CaseNo").distinct().collect()]
    #set table to update
    bronze_table_to_update_df = DeltaTable.forName(spark, bronze_table_to_update)

    #Display Before    
    display(M2_bronze.filter(col("CaseNo").isin(case_nos_to_update)).select("CaseNo", "Appellant_Postcode"))    

    #E3U 5EH
    #SW1A 1A
    # bronze_table_to_update_df.update(
    #     condition=col("CaseNo").isin(case_nos_to_update),
    #     set={
    #         "Appellant_Postcode" : lit("E3U 5EH")                 
    #          }
    # )

    #Display After
    display(M2_bronze.filter(col("CaseNo").isin(case_nos_to_update)).select("CaseNo", "Appellant_Postcode"))    


In [0]:
##############################
#valid_legalrepEmail_not_null - patch one of these : "Rep_Email" , "CaseRep_Email" , "FileSpecificEmail"
##############################


#Setup
checks = {}
bronze_table_to_update = "ariadm_active_appeals.bronze_appealcase_crep_rep_floc_cspon_cfs"
bronze_table = spark.table(bronze_table_to_update)

#Add the Gold Tables to the list below for any States that need updating
state_to_update_1 =  "paymentpending_gold.stg_main_paymentPending_validation"
state_to_update_2 = "paymentpending_gold.stg_main_appeal_submitted_validation"
# states_to_update = [state_to_update_1, state_to_update_2]
states_to_update = [state_to_update_2]



checks["valid_legalrepEmail_not_null"] = "((dv_representation = 'LR' AND legalRepEmail IS NOT NULL AND legalRepEmail RLIKE r'^([a-zA-Z0-9_\\-\\.]+)@([a-zA-Z0-9_\\-\\.]+)\\.([a-zA-Z]{2,5})$') OR (dv_representation != 'LR' AND legalRepEmail IS NULL))"



#####################
#Get Cases Failing Validation
#####################
dq_rules = "({0})".format(" AND ".join(checks.values()))


for state_table in states_to_update:

    df_validation_filtered = spark.sql(f"""
    SELECT *
    FROM {state_table}
    WHERE CaseNo NOT IN (
    SELECT
        CaseNo
    FROM {state_table}
    WHERE {dq_rules})
    """)
    df_validation_filtered = df_validation_filtered.select("CaseNo", "legalRepEmail")


    #####################
    #join with bronze data
    #####################
    #Join Failed Record in Gold With Bronze
    df_validation_filtered = df_validation_filtered.join(M1_bronze, on="CaseNo", how="inner")
    # display(df_validation_filtered)
    # display(df_validation_filtered.select("CaseNo",  "legalRepEmail", "CaseRep_Email"))

    #####################
    #Update Data
    #####################
    #Pull out case Numbers to patch
    case_nos_to_update = [row["CaseNo"] for row in df_validation_filtered.select("CaseNo").distinct().collect()]
    #set table to update
    bronze_table_to_update_df = DeltaTable.forName(spark, bronze_table_to_update)

    #Display Before
    # display(bronze_table)
    display(M1_bronze.filter(col("CaseNo").isin(case_nos_to_update)).select("CaseNo", "CaseRep_Email"))    

    bronze_table_to_update_df.update(
        condition=col("CaseNo").isin(case_nos_to_update),
        set={
           "CaseRep_Email" : lit("joexbloggs@fake.com")                 
            }
    )

    #Display After
    display(M1_bronze.filter(col("CaseNo").isin(case_nos_to_update)).select("CaseNo", "CaseRep_Email"))


In [0]:
##############################
#valid_oocAddressLine1 - patch : "CaseRep_Address1"
##############################


#Setup
checks = {}
bronze_table_to_update = "ariadm_active_appeals.bronze_appealcase_crep_rep_floc_cspon_cfs"

#Add the Gold Tables to the list below for any States that need updating
state_to_update_1 =  "paymentpending_gold.stg_main_paymentPending_validation"
state_to_update_2 = "paymentpending_gold.stg_main_appeal_submitted_validation"
# states_to_update = [state_to_update_1, state_to_update_2]
states_to_update = [state_to_update_2]




checks["valid_oocAddressLine1"] = (
        """(
            (dv_representation = 'LR' AND oocAddressLine1 IS NOT NULL AND legalRepHasAddress <=> 'No')
            OR
            (dv_representation = 'LR' AND oocAddressLine1 IS NULL AND legalRepHasAddress <=> 'Yes')
            OR
            (dv_representation != 'LR' AND oocAddressLine1 IS NULL)
        )"""
    )

#####################
#Get Cases Failing Validation
#####################
dq_rules = "({0})".format(" AND ".join(checks.values()))


for state_table in states_to_update:

    df_validation_filtered = spark.sql(f"""
    SELECT *
    FROM {state_table}
    WHERE CaseNo NOT IN (
    SELECT
        CaseNo
    FROM {state_table}
    WHERE {dq_rules})
    """)
    df_validation_filtered = df_validation_filtered.select("CaseNo", "oocAddressLine1")


    #####################
    #join with bronze data
    #####################
    #Join Failed Record in Gold With Bronze
    df_validation_filtered = df_validation_filtered.join(M1_bronze, on="CaseNo", how="inner")
    display(df_validation_filtered.select("CaseNo",  "oocAddressLine1", "CaseRep_Address1"))

    #####################
    #Update Data
    #####################
    #Pull out case Numbers to patch
    case_nos_to_update = [row["CaseNo"] for row in df_validation_filtered.select("CaseNo").distinct().collect()]
    #set table to update
    bronze_table_to_update_df = DeltaTable.forName(spark, bronze_table_to_update)

    #Display Before    
    display(M1_bronze.filter(col("CaseNo").isin(case_nos_to_update)).select("CaseNo", "CaseRep_Address1"))    

    #E3U 5EH
    #SW1A 1A
    # bronze_table_to_update_df.update(
    #     condition=col("CaseNo").isin(case_nos_to_update),
    #     set={
    #         "CaseRep_Address1" : lit("617 Joshua Park Apt. 191X")                 
    #          }
    # )

    #Display After
    display(M1_bronze.filter(col("CaseNo").isin(case_nos_to_update)).select("CaseNo", "CaseRep_Address1"))    
